In [2]:
from oddt.toolkits import ob
from joblib import delayed, Parallel
from functools import partial
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot as plt
from scipy import stats

import json
import numpy as np

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

PDBBIND_DIR = '/home/fergus/pdbbind/v2019'

with open('../data/pdbbind_training_set.txt') as f:
    train_pdbs = [l.strip() for l in f]
with open('../data/pdbbind_test_set.txt') as f:
    test_pdbs = [l.strip() for l in f]

In [22]:
from oddt.scoring.descriptors.binana import binana_descriptor

@delayed
def build(pdb):
    descriptor_func = binana_descriptor()
    receptor = next(ob.readfile('pdb', f'{PDBBIND_DIR}/{pdb}/{pdb}_protein.pdb'))
    receptor.protein = True
    print(receptor)
    # crystal
    ligand = next(ob.readfile('sdf', f'{PDBBIND_DIR}/{pdb}/{pdb}_ligand.sdf'))
    features_c = descriptor_func.build(ligand, receptor)[0]
    # docked        
    ligand = next(ob.readfile('sdf', f'../pdbbind_docked_poses/{pdb}/{pdb}_ligand_docked.sdf'))
    #features_d = descriptor_func.build(ligand, receptor)[0]
    # docked        
    ligand = next(ob.readfile('sdf', f'../pdbbind_docked_poses/{pdb}/{pdb}_ligand_minimized.sdf'))
    #features_m = descriptor_func.build(ligand, receptor)[0]
    #return (features_c, features_d, features_m)

In [23]:
pdbs = train_pdbs + test_pdbs

with Parallel(n_jobs=6, verbose=10) as parallel:
    features = parallel(build(pdb) for pdb in pdbs)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    1.4s


AttributeError: 'Atom' object has no attribute 'atom_dict'